<h1> Crawling Step 1 - Collecting Data from the flightsradar24 wbesite </h1>

<h2> Preliminary Step </h2>

Let's begin with importing and installing the necessary libraries:

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import time
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import re
import datetime
from datetime import datetime
import numpy as np
from selenium.common.exceptions import ElementNotInteractableException
import warnings
warnings.filterwarnings("ignore")

<h2> The Process </h2>

The data we need to scrape in order to build our desired dataframe is scattered accross the website in different pages, containing lists of flights. 

In these steps, we are loading the webdriver and the webpage, we care accepting cookies and logging in to the user.
after logging in, we want to see all the past 24 hours flights, therefore we are clicking on 'Load earlier flights' button until it is no longer exist

In [2]:
#Opening a new chrome window with Webdriver

driver = webdriver.Chrome(ChromeDriverManager().install())

In [3]:
#inserting the url 
driver.get("https://www.flightradar24.com/data/airports/jfk/departures")

In [4]:
cookie = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Continue")]'))).click()

In [5]:
#The below 3 steps are connecting the user in the website
login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "Log in")]'))).click()

In [6]:
driver.find_element(By.XPATH,"//input[@name=\"email\"]").send_keys("kobyswisa@gmail.com")
driver.find_element(By.XPATH,"//input[@name=\"password\"]").send_keys("Kobkob7570")

In [7]:
Log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Log in")]'))).click()

In [9]:
#after we logged in, we want the webpge to display all 24h past flights.
#this while loop is clicking the 'Load more flight' button until it disappears and we can see 24 hours past flights.

while True:
    try:
        # Find the load button element by its ID attribute
        load_button = driver.find_element(By.CLASS_NAME, 'btn.btn-table-action.btn-flights-load')
        # Click the load button element
        load_button.click()
        # Wait for some time to allow the page to load
        time.sleep(1)
    except ElementNotInteractableException:
        # If the load button element no longer exists, break out of the loop
        break

<h2> Saving flights numbers in a list </h2>


In [10]:
#in the below step, we are saving all the flights numbers in a new list.
#after finished copying all the flights numbers, we are saving the list as a DataFrame.
#Now that we have a DaatFrame, we can remove duplicates and empty cells

flight = driver.find_elements(By.XPATH, '//tbody/tr//td[2]')     

flights_table = []

for i in range(len(flight)):
    temporary_data={'FLIGHT' : flight[i].text}
    flights_table.append(temporary_data)

In [11]:
df_data = pd.DataFrame(flights_table)
df_data

,FLIGHT
0,BR31
1,Y4895
2,3S453
3,CZ600
4,MU7758
...,...
1023,B61518
1024,B61677
1025,B61719
1026,DL1855


In [12]:
df_data = df_data.drop_duplicates()
df_data

,FLIGHT
0,BR31
1,Y4895
2,3S453
3,CZ600
4,MU7758
...,...
1005,3V11
1007,DL3671
1016,AR1301
1018,6R500


In [13]:
df_data = df_data.replace("", np.nan)
df_data

,FLIGHT
0,BR31
1,Y4895
2,3S453
3,CZ600
4,MU7758
...,...
1005,3V11
1007,DL3671
1016,AR1301
1018,6R500


In [14]:
df_data = df_data.dropna()
df_data

,FLIGHT
0,BR31
1,Y4895
2,3S453
3,CZ600
4,MU7758
...,...
1005,3V11
1007,DL3671
1016,AR1301
1018,6R500


In [15]:
df_data.to_csv("flights_list.csv")

<h2> Collecting the data </h2>


In [18]:
#in the below steps, we created a new list called 'flight_departures_JFK', and we are loading in a loop, 
#for each flight number in the flights numbers the webpage, and saving all the columns and lines to 
#the 'flight_departures_JFK' list

flight_departures_JFK = []
for i in df_data["FLIGHT"]:
    full_url = "https://www.flightradar24.com/data/flights/" + i
    try:
        driver.get(full_url)
        while True:
            try:
                load_button = driver.find_element(By.ID, 'btn-load-earlier-flights')
                if load_button.is_enabled():
                    load_button.click()
                    # Wait for some time to allow the page to load
                    time.sleep(1)
                else:
                    break
            except NoSuchElementException:
                # If the load button element no longer exists, break out of the loop
                break
        date = driver.find_elements(By.XPATH, '//tbody/tr//td[3]')
        From = driver.find_elements(By.XPATH, '//tbody/tr//td[4]')
        to = driver.find_elements(By.XPATH, '//tbody/tr//td[5]')
        aircraft = driver.find_elements(By.XPATH, '//tbody/tr//td[6]')
        flight_time = driver.find_elements(By.XPATH, '//tbody/tr//td[7]')
        std = driver.find_elements(By.XPATH, '//tbody/tr//td[8]')
        atd = driver.find_elements(By.XPATH, '//tbody/tr//td[9]')
        sta = driver.find_elements(By.XPATH, '//tbody/tr//td[10]')
        status = driver.find_elements(By.XPATH, '//tbody/tr//td[12]')
        for i in range(len(date)):
            temporary_data={'DATE' : date[i].text, 'FROM' : From[i].text, 'TO' : to[i].text, 'AIRCRAFT' : aircraft[i].text, 
            'FLIGHT TIME' : flight_time[i].text, 'STD' : std[i].text, 'ATD' : atd[i].text, 'STA' : sta[i].text, 
            'STATUS' : status[i].text,}
            flight_arrivals_JFK.append(temporary_data)
    except:
        print(f"Error occurred while processing {i}. Breaking the loop.")
        continue

In [17]:
##saving the list to a new DataFrame called data_df2

df_data2 = pd.DataFrame(flight_departures_JFK)
df_data2

,DATE,FROM,TO,AIRCRAFT,FLIGHT TIME,STD,ATD,STA,STATUS
0,15 May 2023,New York (JFK),Taipei (TPE),77W,—,01:25,—,05:15,Scheduled
1,14 May 2023,New York (JFK),Taipei (TPE),77W,—,01:25,—,05:15,Scheduled
2,13 May 2023,New York (JFK),Taipei (TPE),77W,—,01:25,—,05:15,Scheduled
3,12 May 2023,New York (JFK),Taipei (TPE),77W,—,01:25,—,05:15,Scheduled
4,11 May 2023,New York (JFK),Taipei (TPE),77W,—,01:25,—,05:15,Scheduled
...,...,...,...,...,...,...,...,...,...
63215,29 Jan 2023,Mexico City (MEX),New York (JFK),A306 (XA-UYR),3:51,08:15,01:44,14:00,Landed 06:35
63216,22 Jan 2023,Mexico City (MEX),New York (JFK),B762 (XA-EFR),—,08:15,18:37,14:00,Diverted to ORD
63217,19 Jan 2023,New York (JFK),Chicago (ORD),B762 (XA-LRC),2:04,16:55,17:04,18:05,Landed 18:08
63218,19 Jan 2023,Guatemala City (GUA),New York (JFK),B762 (XA-LRC),4:13,10:30,09:21,15:25,Landed 14:34


In [18]:
#Finally, after finished collecting the data, we are saving the DataFrame as a csv file
#in order to clean it in the Data Handling notebook

df_data2.to_csv("flight_departures_JFK_raw_data.csv", index=False)